# Creación dataset modelo 1. Infraestructuras para deep learning.
El objetivo de este notebook es la creación del dataset para el modelo 1 de IPDL. En la asignatura de Machine Learning (práctica 2) se planteó el problema de predicción de series temporales para datos de calidad del aire de la Aljorra. Aunque no se usaron redes neuronales para dichas predicciones vamos a proponer un caso de uso sencillo en el que se use este dataset e inspirado en esta práctica. **Imaginemos que el sensor de calidad del aire de la Aljorra sufre de un fallo que hace que a veces no reporte datos y estamos muy interesados de que estos problemas se arreglen en el Edge para que los usuarios finales del dashboard de reporte no encuentren datos en blanco. De esta manera, se propone crear un modelo mediante un perceptrón multicapa que, cuando sucedan estos fallos, sea capaz de predecir el valor faltante usando como entrada los 12 valores anteriormente reportados por el sensor.** Los datos utilizados para el entrenamiento y test del modelo serán los del año 2022 y se generarán lags con los 12 pasos de tiempo anterior y la variable target.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
!wget https://umubox.um.es/index.php/s/sMMVquTTFtqHZiG/download -O air_quality.zip

--2025-03-15 12:15:51--  https://umubox.um.es/index.php/s/sMMVquTTFtqHZiG/download
Resolving umubox.um.es (umubox.um.es)... 155.54.212.96, 2001:720:1710:212::1:b
Connecting to umubox.um.es (umubox.um.es)|155.54.212.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2475459 (2.4M) [application/zip]
Saving to: ‘air_quality.zip’

air_quality.zip     100%[===================>]   2.36M  2.05MB/s    in 1.2s    

2025-03-15 12:15:53 (2.05 MB/s) - ‘air_quality.zip’ saved [2475459/2475459]



In [3]:
!unzip -o air_quality.zip

Archive:  air_quality.zip
   creating: air_quality/
  inflating: air_quality/Aljorrahorarias2017.xls  
  inflating: air_quality/Aljorrahorarias2018.xls  
  inflating: air_quality/Aljorrahorarias2019.xls  
  inflating: air_quality/Aljorrahorarias2020.xls  
  inflating: air_quality/Aljorrahorarias2021.xls  
  inflating: air_quality/Aljorrahorarias2022.xls  


In [4]:
files_dir = "/content/air_quality"

files_to_import = [os.path.join(files_dir, file) for file in os.listdir(files_dir)]

files_to_import

['/content/air_quality/Aljorrahorarias2017.xls',
 '/content/air_quality/Aljorrahorarias2020.xls',
 '/content/air_quality/Aljorrahorarias2018.xls',
 '/content/air_quality/Aljorrahorarias2019.xls',
 '/content/air_quality/Aljorrahorarias2021.xls',
 '/content/air_quality/Aljorrahorarias2022.xls']

In [5]:
import pandas as pd
import os

# Assuming files_to_import is defined as in the previous code
columnas = ["Fecha", "NO (µg/m³N)", "NO2 (µg/m³N)", "SO2 (µg/m³N)", "O3 (µg/m³N)", "TMP (ºC)", "HR (% H.R.)", "NOX (µg/m³N)", "DD (grados)", "PRB (mb)", "RS (W/m³)", "VV (m/s)", "C6H6 (µg/m³N)", 	"C7H8 (µg/m³N)",
                                      "XIL (µg/m³N)", "PM10 (µg/m³N)", "Ruido (dBA)"]
df_complete = pd.DataFrame(columns = columnas )

for file in files_to_import:
    if file.endswith(".xls") or file.endswith(".xlsx"):  # Check for Excel files
        try:
            df = pd.read_excel(file, usecols = columnas, engine= "xlrd", skiprows=1)  # Read each Excel file
            df_complete = pd.concat([df_complete, df], ignore_index=True) # Append to all_data
        except Exception as e:
            print(f"Error reading file {file}: {e}")
    else:
        print(f"Skipping file {file} (not an Excel file)")

df_complete

*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


<ipython-input-5-6f03a2f7bf50>:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_complete = pd.concat([df_complete, df], ignore_index=True) # Append to all_data


*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


,Fecha,NO (µg/m³N),NO2 (µg/m³N),SO2 (µg/m³N),O3 (µg/m³N),TMP (ºC),HR (% H.R.),NOX (µg/m³N),DD (grados),PRB (mb),RS (W/m³),VV (m/s),C6H6 (µg/m³N),C7H8 (µg/m³N),XIL (µg/m³N),PM10 (µg/m³N),Ruido (dBA)
0,1 ene. 2017 00:01,5.0,30.0,15.0,43.0,10.0,85.0,37.0,299.0,1031.0,2.0,1.0,NaN,NaN,NaN,26.00,NaN
1,1 ene. 2017 01:01,4.0,27.0,12.0,41.0,9.0,84.0,34.0,300.0,1030.0,2.0,1.0,NaN,NaN,NaN,25.00,NaN
2,1 ene. 2017 02:01,5.0,25.0,14.0,39.0,9.0,86.0,33.0,300.0,1030.0,2.0,1.0,NaN,NaN,NaN,21.00,NaN
3,1 ene. 2017 03:01,3.0,16.0,15.0,43.0,8.0,85.0,21.0,301.0,1029.0,3.0,1.0,NaN,NaN,NaN,18.00,NaN
4,1 ene. 2017 04:01,4.0,15.0,14.0,46.0,8.0,84.0,22.0,302.0,1029.0,2.0,1.0,NaN,NaN,NaN,16.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52579,31 dic. 2022 19:12,2.5,10.0,4.6,57.6,16.0,77.0,13.9,185.0,1024.0,2.0,1.2,NaN,NaN,NaN,31.75,NaN
52580,31 dic. 2022 20:12,4.6,17.5,5.0,28.2,14.0,81.0,24.6,235.0,1025.0,2.0,1.0,NaN,NaN,NaN,53.27,NaN
52581,31 dic. 2022 21:12,3.9,12.9,4.9,27.4,13.0,88.0,18.8,280.0,1025.0,2.0,1.0,NaN,NaN,NaN,55.28,NaN
52582,31 dic. 2022 22:12,6.3,18.4,5.2,23.2,12.0,88.0,28.0,251.0,1025.0,3.0,1.0,NaN,NaN,NaN,64.92,NaN


In [6]:
map_month_dict = {
    "ene.": "/01/",
    "feb.": "/02/",
    "mar.": "/03/",
    "abr.": "/04/",
    "may.": "/05/",
    "jun.": "/06/",
    "jul.": "/07/",
    "ago.": "/08/",
    "sep.": "/09/",
    "oct.": "/10/",
    "nov.": "/11/",
    "dic.": "/12/"
}


In [7]:
from datetime import datetime


def map_to_datetime(date_str):
    # Divide la fecha en día, mes, año y hora
    day, month_abbr, year_time = date_str.split(" ", 2)
    year, time = year_time.split(" ", 1)

    # Reemplaza el mes abreviado por su equivalente numérico
    month_numeric = map_month_dict[month_abbr]
    date_formatted = f"{day}{month_numeric}{year} {time}"

    # Convierte al formato datetime
    return datetime.strptime(date_formatted, "%d/%m/%Y %H:%M")

In [8]:
df_complete["Fecha"] = df_complete["Fecha"].map(map_to_datetime)


In [9]:
df_complete.sort_values(by="Fecha", inplace=True)

In [10]:
df_complete

,Fecha,NO (µg/m³N),NO2 (µg/m³N),SO2 (µg/m³N),O3 (µg/m³N),TMP (ºC),HR (% H.R.),NOX (µg/m³N),DD (grados),PRB (mb),RS (W/m³),VV (m/s),C6H6 (µg/m³N),C7H8 (µg/m³N),XIL (µg/m³N),PM10 (µg/m³N),Ruido (dBA)
0,2017-01-01 00:01:00,5.0,30.0,15.0,43.0,10.0,85.0,37.0,299.0,1031.0,2.0,1.0,NaN,NaN,NaN,26.00,NaN
1,2017-01-01 01:01:00,4.0,27.0,12.0,41.0,9.0,84.0,34.0,300.0,1030.0,2.0,1.0,NaN,NaN,NaN,25.00,NaN
2,2017-01-01 02:01:00,5.0,25.0,14.0,39.0,9.0,86.0,33.0,300.0,1030.0,2.0,1.0,NaN,NaN,NaN,21.00,NaN
3,2017-01-01 03:01:00,3.0,16.0,15.0,43.0,8.0,85.0,21.0,301.0,1029.0,3.0,1.0,NaN,NaN,NaN,18.00,NaN
4,2017-01-01 04:01:00,4.0,15.0,14.0,46.0,8.0,84.0,22.0,302.0,1029.0,2.0,1.0,NaN,NaN,NaN,16.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52579,2022-12-31 19:12:00,2.5,10.0,4.6,57.6,16.0,77.0,13.9,185.0,1024.0,2.0,1.2,NaN,NaN,NaN,31.75,NaN
52580,2022-12-31 20:12:00,4.6,17.5,5.0,28.2,14.0,81.0,24.6,235.0,1025.0,2.0,1.0,NaN,NaN,NaN,53.27,NaN
52581,2022-12-31 21:12:00,3.9,12.9,4.9,27.4,13.0,88.0,18.8,280.0,1025.0,2.0,1.0,NaN,NaN,NaN,55.28,NaN
52582,2022-12-31 22:12:00,6.3,18.4,5.2,23.2,12.0,88.0,28.0,251.0,1025.0,3.0,1.0,NaN,NaN,NaN,64.92,NaN


Nos quedamos solo con las fechas de 2022 para el entrenamiento y ponemos granularidad horaria

In [11]:
df_sample = df_complete[df_complete["Fecha"].dt.year == 2022]
df_sample = df_sample.set_index("Fecha")
df_sample = df_sample.resample("H").mean()
df_sample

<ipython-input-11-75f77aac2dc5>:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_sample = df_sample.resample("H").mean()


,NO (µg/m³N),NO2 (µg/m³N),SO2 (µg/m³N),O3 (µg/m³N),TMP (ºC),HR (% H.R.),NOX (µg/m³N),DD (grados),PRB (mb),RS (W/m³),VV (m/s),C6H6 (µg/m³N),C7H8 (µg/m³N),XIL (µg/m³N),PM10 (µg/m³N),Ruido (dBA)
Fecha,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,2.0,11.0,NaN,8.0,9.0,100.0,15.0,318.0,1027.0,3.0,1.0,NaN,NaN,NaN,26.00,NaN
2022-01-01 01:00:00,5.0,12.0,NaN,4.0,9.0,100.0,19.0,245.0,1027.0,4.0,1.0,NaN,NaN,NaN,31.00,NaN
2022-01-01 02:00:00,7.0,13.0,NaN,3.0,9.0,100.0,24.0,302.0,1027.0,4.0,1.0,NaN,NaN,NaN,32.00,NaN
2022-01-01 03:00:00,2.0,11.0,NaN,6.0,8.0,100.0,15.0,263.0,1027.0,4.0,1.0,NaN,NaN,NaN,30.00,NaN
2022-01-01 04:00:00,3.0,11.0,NaN,3.0,8.0,100.0,15.0,1.0,1027.0,5.0,1.0,NaN,NaN,NaN,41.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,2.5,10.0,4.6,57.6,16.0,77.0,13.9,185.0,1024.0,2.0,1.2,NaN,NaN,NaN,31.75,NaN
2022-12-31 20:00:00,4.6,17.5,5.0,28.2,14.0,81.0,24.6,235.0,1025.0,2.0,1.0,NaN,NaN,NaN,53.27,NaN
2022-12-31 21:00:00,3.9,12.9,4.9,27.4,13.0,88.0,18.8,280.0,1025.0,2.0,1.0,NaN,NaN,NaN,55.28,NaN


In [12]:
NOX_data = list(df_sample["NOX (µg/m³N)"])

In [13]:
len(NOX_data)

8760

In [14]:
# Número de lags
n_lags = 12

# Crear listas para el dataframe
data = []
for i in range(len(NOX_data) - n_lags):
    row = NOX_data[i:i + n_lags] + [NOX_data[i + n_lags]]
    data.append(row)

# Crear dataframe
columns = [f'NOX_level_prev{n_lags - i }' for i in range(n_lags)] + ['target']
lags_df = pd.DataFrame(data, columns=columns)

lags_df

,NOX_level_prev12,NOX_level_prev11,NOX_level_prev10,NOX_level_prev9,NOX_level_prev8,NOX_level_prev7,NOX_level_prev6,NOX_level_prev5,NOX_level_prev4,NOX_level_prev3,NOX_level_prev2,NOX_level_prev1,target
0,15.0,19.0,24.0,15.0,15.0,13.0,12.0,13.0,11.0,12.0,15.0,14.0,13.0
1,19.0,24.0,15.0,15.0,13.0,12.0,13.0,11.0,12.0,15.0,14.0,13.0,13.0
2,24.0,15.0,15.0,13.0,12.0,13.0,11.0,12.0,15.0,14.0,13.0,13.0,14.0
3,15.0,15.0,13.0,12.0,13.0,11.0,12.0,15.0,14.0,13.0,13.0,14.0,13.0
4,15.0,13.0,12.0,13.0,11.0,12.0,15.0,14.0,13.0,13.0,14.0,13.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8743,10.6,14.8,15.5,15.8,12.9,14.1,12.8,14.6,12.2,11.4,11.6,10.6,13.9
8744,14.8,15.5,15.8,12.9,14.1,12.8,14.6,12.2,11.4,11.6,10.6,13.9,24.6
8745,15.5,15.8,12.9,14.1,12.8,14.6,12.2,11.4,11.6,10.6,13.9,24.6,18.8
8746,15.8,12.9,14.1,12.8,14.6,12.2,11.4,11.6,10.6,13.9,24.6,18.8,28.0


In [15]:
lags_df.to_csv("air_quality_2022_laAljorra.csv", index=False)

## Datos para la inferencia

In [16]:
df_sample = df_complete[
    (df_complete["Fecha"].dt.year >= 2020) &
    (df_complete["Fecha"].dt.year <= 2021)
]

df_sample = df_sample.set_index("Fecha")
df_sample = df_sample.resample("H").mean()
df_sample

<ipython-input-16-668cc9682a99>:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_sample = df_sample.resample("H").mean()


,NO (µg/m³N),NO2 (µg/m³N),SO2 (µg/m³N),O3 (µg/m³N),TMP (ºC),HR (% H.R.),NOX (µg/m³N),DD (grados),PRB (mb),RS (W/m³),VV (m/s),C6H6 (µg/m³N),C7H8 (µg/m³N),XIL (µg/m³N),PM10 (µg/m³N),Ruido (dBA)
Fecha,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,1.0,14.0,8.0,17.0,9.0,100.0,16.0,289.0,1032.0,3.0,1.0,0.5,0.8,0.7,22.0,NaN
2020-01-01 01:00:00,3.0,16.0,6.0,15.0,8.0,100.0,20.0,318.0,1032.0,3.0,1.0,0.5,0.8,0.6,19.0,NaN
2020-01-01 02:00:00,2.0,13.0,7.0,35.0,10.0,100.0,16.0,297.0,1031.0,4.0,1.0,0.4,0.7,0.9,30.0,NaN
2020-01-01 03:00:00,1.0,9.0,8.0,32.0,9.0,100.0,11.0,295.0,1031.0,3.0,1.0,0.3,0.5,1.5,13.0,NaN
2020-01-01 04:00:00,1.0,8.0,7.0,32.0,8.0,100.0,10.0,297.0,1031.0,3.0,1.0,0.4,0.5,1.4,17.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00,4.0,15.0,NaN,24.0,15.0,87.0,21.0,164.0,1026.0,1.0,1.0,NaN,NaN,NaN,34.0,NaN
2021-12-31 20:00:00,3.0,15.0,NaN,14.0,13.0,97.0,20.0,220.0,1026.0,1.0,1.0,NaN,NaN,NaN,26.0,NaN
2021-12-31 21:00:00,4.0,17.0,NaN,5.0,12.0,100.0,24.0,306.0,1027.0,2.0,1.0,NaN,NaN,NaN,44.0,NaN


In [17]:
NOX_data = list(df_sample["NOX (µg/m³N)"])

In [18]:
len(NOX_data)

17544

In [19]:
# Número de lags
n_lags = 12

# Crear listas para el dataframe
data = []
for i in range(len(NOX_data) - n_lags):
    row = NOX_data[i:i + n_lags] + [NOX_data[i + n_lags]]
    data.append(row)

# Crear dataframe
columns = [f'NOX_level_prev{n_lags - i }' for i in range(n_lags)] + ['target']
lags_df = pd.DataFrame(data, columns=columns)

lags_df

,NOX_level_prev12,NOX_level_prev11,NOX_level_prev10,NOX_level_prev9,NOX_level_prev8,NOX_level_prev7,NOX_level_prev6,NOX_level_prev5,NOX_level_prev4,NOX_level_prev3,NOX_level_prev2,NOX_level_prev1,target
0,16.0,20.0,16.0,11.0,10.0,11.0,8.0,10.0,11.0,10.0,10.0,14.0,16.0
1,20.0,16.0,11.0,10.0,11.0,8.0,10.0,11.0,10.0,10.0,14.0,16.0,12.0
2,16.0,11.0,10.0,11.0,8.0,10.0,11.0,10.0,10.0,14.0,16.0,12.0,13.0
3,11.0,10.0,11.0,8.0,10.0,11.0,10.0,10.0,14.0,16.0,12.0,13.0,11.0
4,10.0,11.0,8.0,10.0,11.0,10.0,10.0,14.0,16.0,12.0,13.0,11.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527,17.0,19.0,48.0,32.0,29.0,23.0,20.0,16.0,16.0,14.0,16.0,17.0,21.0
17528,19.0,48.0,32.0,29.0,23.0,20.0,16.0,16.0,14.0,16.0,17.0,21.0,20.0
17529,48.0,32.0,29.0,23.0,20.0,16.0,16.0,14.0,16.0,17.0,21.0,20.0,24.0
17530,32.0,29.0,23.0,20.0,16.0,16.0,14.0,16.0,17.0,21.0,20.0,24.0,22.0


In [20]:
lags_df.to_csv("air_quality_20202021_inference_laAljorra.csv", index=False)